# Using Web Processing Service (WPS) with the Defra Earth Observation Data Service API
This notebook introduces the concept of the Web Processing Service (WPS) which enables users to submit instructions to the EO Data Service to return outputs. The instructions are contained in the XML files provided.  Please ensure these are saved in the XML folder within the Scripts folder.

## Instructions:
1. Select one of the eight WPS processes to run by typing its name in the relevant cell.
2. Select 'Restart and Clear Output' from the Kernel menu at the top of this page if necessary.
3. Select 'Run All' from the Cell menu at the top of this page.
4. Look at the outputs beneath the final cell.  You should see the following, preceded by a date-time stamp:
        -WPS request submitted
        -WPS request response code (200) and the full WPS response including executionId number
        -exececutionId extracted from the WPS request response code 
        -The status check URL including the executionId. This will run automatically to check the status of your request.
        -Status check results every 15 seconds: process is still running or result is ready for download. 
        -Download URL
5. Click on the download URL or copy and paste it into your browser.
6. The downloaded file should be saved to your default download location.
7. Check the downloaded file, e.g. open spatial datasets in a GIS application. 
8. Now choose one of the other WPS processes (step 1) and repeat the above steps until you have tested them all.
        


In [ ]:
import requests
import os
from IPython.display import Image
import time
from datetime import datetime
import importlib
import urllib3
import config

In [ ]:
importlib.reload(config)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
output_fmt='%Y%m%dT%H%M%S'
pretty_fmt='%Y-%m-%d %H:%M:%S'
wps_server = config.URL + config.WPS_SUF + '?access_token=' + config.ACCESS_TOKEN
headers = {'Content-type': 'application/xml','User-Agent': 'curl/7.65.1'}

wps_test_config = {
    'bandselect':['bandselect_EODS-sp14.xml','image/tiff','result.tiff'],
    'rasterclip':['rasterclip_EODS-sp14.xml','image/tiff','result.tiff'],
    'zonalstats':['zonalstats_EODS.xml','text/csv','statistics.csv'],
    'rastertopoints':['raster-to-points_EODS-sp14.xml','application/zip','result.zip'],
    'coverageclassstats':['coverageclassstats_EODS-sp14.xml','text/xml','result.xml'],
    'reproject':['reproject_EODS-sp14.xml','application/zip','result.zip'],
    'gsdownload-small':['gsdownload-EODS-small-s2.xml','application/zip','result.zip'],
    'gsdownload-large':['gsdownload-EODS-large-s1.xml','application/zip','result.zip'],
    }

# Select the WPS "Tool" to Run

### The options are:

* 'bandselect' (WPS tool name: ras:BandSelect) - selects a single band from a raster
* 'rasterclip' (WPS tool name: ras:CropCoverage) - clips an area of a raster defined by geometry provided in well-known text (WKT) format.
* 'zonalstats' (WPS tool name: ras:RasterZonalStatistics) - generates zonal statistics from a raster using geometry supplied as a shapefile.
* 'rastertopoints' (WPS tool name: gs:RasterAsPointCollection) - generates a point for each valid pixel in a raster dataset.  Band values are stored as attributes of the points.
* 'coverageclassstats' (WPS tool name: ras:CoverageClassStats) - NOT CURRENTLY WORKING but should calculate statistics from raster values classified into bins/classes (i.e. a histogram).
* 'reproject' (WPS tool name: gs:reproject) - reprojects a vector dataset into a supplied coordinate reference system.
* 'gsdownload-small' (WPS tool name: gs:download) - downloads a single layer, in this case a small S2 granule.
* 'gsdownload-large' (WPS tool name: gs:download) - downloads a single layer, in this case a large S1 scene.


In [ ]:
wps_tool = 'bandselect'

In [ ]:
# STEP 1 of 3. Submit WPS "Process submission" request

# get the configuration for the wps tool to run based in user input
xml_payload_file = open(os.path.join(os.path.join(os.getcwd(),'xml'),wps_test_config.get(wps_tool)[0]),'r')
mime_type = wps_test_config.get(wps_tool)[1]
output_id = wps_test_config.get(wps_tool)[2]

# set file extension based on mime type
print('\n' + datetime.utcnow().strftime(pretty_fmt) + ' :: the wps request was submitted to the address :: ' + wps_server)
wps_submit_response = requests.post(wps_server, data=xml_payload_file.read(), headers=headers, verify=True)
status_code = wps_submit_response.status_code
print('\n' + datetime.utcnow().strftime(pretty_fmt)
      + ' :: the wps request response code is: "' + str(status_code) + '" and the wps request response is \n'
      + wps_submit_response.text)

# if connection to the WPS server was successfully, check progress and download result
if status_code == 200 and not wps_submit_response.text.find('ExceptionReport') > 0:
    execution_id = wps_submit_response.text.split('executionId=')[1].split('&')[0]
    print('\n' + datetime.utcnow().strftime(pretty_fmt) + ' :: the WPS execution_id is ' + str(execution_id))
       
    for ii in range(99):

        # STEP 2 of 3. Submit WPS "STATUS CHECK" request, depending on the size of the wps job, this could take time        
        status_check_url = wps_server + '&request=GetExecutionStatus&executionid=' + execution_id
        if ii == 0:
            print('\n' + datetime.utcnow().strftime(pretty_fmt) + ' :: status check url is ' + status_check_url)
        status_response = requests.get(status_check_url,headers=headers)
        print('\n' + datetime.utcnow().strftime(pretty_fmt) 
              + ' :: Status Check/Download Attempt No: ' + str(ii) 
              + ' :: process currently running on wps server, please wait')

        if status_response.text.find('wps:ProcessFailed') != -1:
            print('\n' + datetime.utcnow().strftime(pretty_fmt) 
                  + ' :: WPS Processed Failed, check the status URL for the specific error = ' + status_check_url)
            break
        
        # STEP 3 of 3. if the download is ready with 'Process succeeded' message, then print the download URL to the output
        if status_response.text.find('wps:ProcessSucceeded') != -1:
            result_url = wps_server + '&request=GetExecutionResult&executionid=' + execution_id + '&outputId=' + output_id +'&mimetype=' + mime_type
            print('\n' + datetime.utcnow().strftime(pretty_fmt) 
                  + ' :: Result is ready for download on URL = ' + result_url)
            break

        # wait 15 seconds between checks 
        time.sleep(15)

else:        
    # if WPS request was not successful, quit workflow
    print('\n' + datetime.utcnow().strftime(pretty_fmt) + ' the WPS request submission was not successful, quitting workflow')

## Thank you for your help with testing the Earth Observation Data Service API and WPS.